# Dataset Overview — RetailRocket

Цель ноутбука:
- ознакомиться со структурой датасета RetailRocket,
- оценить размеры данных,
- изучить распределение типов событий,
- получить первые базовые инсайты перед EDA и моделированием.


In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 50)


In [4]:
DATA_PATH = "../data/raw/events.csv"

events = pd.read_csv(DATA_PATH)
events.head()


,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [5]:
events.info()


<class 'pandas.DataFrame'>
RangeIndex: 2756101 entries, 0 to 2756100
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   timestamp      int64  
 1   visitorid      int64  
 2   event          str    
 3   itemid         int64  
 4   transactionid  float64
dtypes: float64(1), int64(3), str(1)
memory usage: 105.1 MB


In [6]:
events["timestamp"] = pd.to_datetime(events["timestamp"], unit="ms")
events["date"] = events["timestamp"].dt.date
events["hour"] = events["timestamp"].dt.hour


In [7]:
print("Total events:", len(events))
print("Unique users:", events["visitorid"].nunique())
print("Unique items:", events["itemid"].nunique())


Total events: 2756101
Unique users: 1407580
Unique items: 235061


In [8]:
events["event"].value_counts()


event
view           2664312
addtocart        69332
transaction      22457
Name: count, dtype: int64

In [9]:
events["event"].value_counts(normalize=True)


event
view           0.966696
addtocart      0.025156
transaction    0.008148
Name: proportion, dtype: float64

In [10]:
user_activity = events.groupby("visitorid").size()

user_activity.describe()


count    1.407580e+06
mean     1.958042e+00
std      1.258049e+01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      7.757000e+03
dtype: float64

In [11]:
user_activity.quantile([0.5, 0.9, 0.99])


0.50     1.0
0.90     3.0
0.99    13.0
dtype: float64

In [12]:
item_popularity = events.groupby("itemid").size()

item_popularity.describe()


count    235061.000000
mean         11.725046
std          37.032101
min           1.000000
25%           1.000000
50%           3.000000
75%           9.000000
max        3412.000000
dtype: float64

In [13]:
item_popularity.quantile([0.5, 0.9, 0.99])


0.50      3.0
0.90     25.0
0.99    143.0
dtype: float64

In [14]:
events["timestamp"] = pd.to_datetime(events["timestamp"], unit="ms")
events["date"] = events["timestamp"].dt.date
events["hour"] = events["timestamp"].dt.hour

events["timestamp"].min(), events["timestamp"].max()


(Timestamp('2015-05-03 03:00:04.384000'),
 Timestamp('2015-09-18 02:59:47.788000'))

- Данные покрывают период с `2015-05-03 03:00:04.384000` по `2015-09-18 02:59:47.788000` .
- Это позволяет в дальнейшем делать временной train/validation/test сплит.


## Первичные выводы по датасету RetailRocket

- Объём данных:
  - ~2.76M событий,
  - ~1.41M уникальных пользователей,
  - ~235K уникальных товаров.

- Типы событий:
  - ~96.7% — просмотры (`view`),
  - ~2.5% — добавления в корзину (`addtocart`),
  - ~0.8% — покупки (`transaction`).
  → покупки — редкий, но ключевой сигнал, под котором мы будем оценивать рекомендации.

- Активность пользователей:
  - медиана = 1 событие на пользователя,
  - 90-й перцентиль = 3 события,
  - 99-й перцентиль = 13 событий,
  - максимум ≈ 7.7K событий.
  → выраженный long-tail: очень много одноразовых посетителей и небольшой хвост супер-активных.

- Популярность товаров:
  - медиана = 3 события на товар,
  - 90-й перцентиль = 25,
  - 99-й перцентиль = 143,
  - максимум ≈ 3.4K событий.
  → большинство товаров очень редко взаимодействуют с пользователями, есть небольшой набор хитов.

**Вывод:** датасет типичен для e-commerce:
- редкие покупки,
- много одноразовых пользователей,
- long-tail по товарам.

Для построения рекомендательной системы потребуется:
- фильтрация редких пользователей и товаров,
- аккуратный временной сплит (train/val/test) на основе `timestamp`.
